In [7]:
from pylate import indexes, models, retrieve

model = models.ColBERT(
    model_name_or_path="jinaai/jina-colbert-v2",trust_remote_code=True,
    device="cuda"
)

index = indexes.Voyager(
    index_folder="pylate-index",
    index_name="index",
    override=True,
)

retriever = retrieve.ColBERT(index=index)

No sentence-transformers model found with name jinaai/jina-colbert-v2.
Loaded the ColBERT model from Stanford NLP.
The tokenizer does not support resizing the token embeddings, the prefixes token have not been added to vocabulary.


In [8]:
from datasets import load_dataset
ds = load_dataset('arbml/Hadith')

In [11]:
documents = ds['train']['Text'][:1000]
documents_ids = [str(i) for i in range(len(documents))]

In [12]:


# Encode the documents
documents_embeddings = model.encode(
    documents,
    batch_size=32,
    is_query=False, # Encoding documents
    show_progress_bar=True,
    device="cuda"
)

# Add the documents ids and embeddings to the Voyager index
index.add_documents(
    documents_ids=documents_ids,
    documents_embeddings=documents_embeddings,
)

Encoding documents (bs=32):   0%|          | 0/32 [00:00<?, ?it/s]

Adding documents to the index (bs=2000): 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


In [13]:
print(documents[0])
print(documents[1])
print(documents[2])


أخبرنا قتيبة بن سعيد قال حدثنا سفيان عن الزهري عن أبي سلمة عن أبي هريرة أن النبي صلى الله عليه وسلم قال إذا استيقظ أحدكم من نومه فلا يغمس يده في وضوئه حتى يغسلها ثلاثا فإن أحدكم لا يدري أين باتت يده.
أخبرنا إسحق بن إبراهيم وقتيبة بن سعيد عن جرير عن منصور عن أبي وائل عن حذيفة قال كان رسول الله صلى الله عليه وسلم إذا قام من الليل يشوص فاه بالسواك.
أخبرنا أحمد بن عبدة قال حدثنا حماد بن زيد قال أخبرنا غيلان بن جرير عن أبي بردة عن أبي موسى قال دخلت على رسول الله صلى الله عليه وسلم وهو يستن وطرف السواك على لسانه وهو يقول عأ عأ.


In [41]:
queries_embeddings = model.encode(
    ["اذا استيقظ احدكم من نومه فليسغل يده","لو قام من الليل فليشوص فاه بالسواك","يستن بطرف السواك"],

    batch_size=32,
    is_query=True, # Encoding queries
    show_progress_bar=True,
)

scores = retriever.retrieve(
    queries_embeddings=queries_embeddings, 
    k=10,
)

print(scores)

Encoding queries (bs=32):   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving documents (bs=50): 100%|██████████| 1/1 [00:00<00:00,  1.35it/s]

[[{'id': '0', 'score': 13.545108795166016}, {'id': '160', 'score': 13.0540771484375}, {'id': '88', 'score': 12.205108642578125}, {'id': '436', 'score': 11.819663047790527}, {'id': '610', 'score': 11.20112419128418}, {'id': '612', 'score': 10.773809432983398}, {'id': '255', 'score': 10.679129600524902}, {'id': '440', 'score': 10.675375938415527}, {'id': '256', 'score': 10.647441864013672}, {'id': '836', 'score': 10.599963188171387}], [{'id': '1', 'score': 13.380562782287598}, {'id': '436', 'score': 9.462864875793457}, {'id': '6', 'score': 9.286032676696777}, {'id': '616', 'score': 9.073838233947754}, {'id': '185', 'score': 9.03082275390625}, {'id': '836', 'score': 8.95811939239502}, {'id': '796', 'score': 8.951371192932129}, {'id': '88', 'score': 8.922164916992188}, {'id': '259', 'score': 8.829508781433105}, {'id': '619', 'score': 8.804692268371582}], [{'id': '2', 'score': 7.123141288757324}, {'id': '5', 'score': 5.21481990814209}, {'id': '1', 'score': 5.0685200691223145}, {'id': '7', '

In [43]:
scores
for score in scores:
    # print(score)
    for el in score:
        item = int(el['id'])
        print(documents[item])


أخبرنا قتيبة بن سعيد قال حدثنا سفيان عن الزهري عن أبي سلمة عن أبي هريرة أن النبي صلى الله عليه وسلم قال إذا استيقظ أحدكم من نومه فلا يغمس يده في وضوئه حتى يغسلها ثلاثا فإن أحدكم لا يدري أين باتت يده.
أخبرنا إسمعيل بن مسعود وحميد بن مسعدة قالا حدثنا يزيد بن زريع قال حدثنا معمر عن الزهري عن أبي سلمة عن أبي هريرة أن رسول الله صلى الله عليه وسلم قال إذا استيقظ أحدكم من منامه فلا يدخل يده في الإناء حتى يفرغ عليها ثلاث مرات فإنه لا يدري أين باتت يده.
أخبرنا محمد بن زنبور المكي قال حدثنا ابن أبي حازم عن يزيد بن عبد الله أن محمد بن إبراهيم حدثه عن عيسى بن طلحة عن أبي هريرة عن رسول الله صلى الله عليه وسلم قال إذا استيقظ أحدكم من منامه فتوضأ فليستنثر ثلاث مرات فإن الشيطان يبيت على خيشومه.
أخبرنا عمران بن يزيد قال حدثنا إسمعيل بن عبد الله قال حدثنا الأوزاعي قال حدثنا محمد بن مسلم الزهري قال حدثني سعيد بن المسيب قال حدثني أبو هريرة قال قال رسول الله صلى الله عليه وسلم إذا قام أحدكم من الليل فلا يدخل يده في الإناء حتى يفرغ عليها مرتين أو ثلاثا فإن أحدكم لا يدري أين باتت يده.
أخبرنا قتيبة قال حدثنا 